# Training Time

In [1]:
import glob
from pprint import pprint

import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from slice_generator import slice_generator

In [2]:
print("tensorflow version:",tf.__version__)

tensorflow version: 2.2.0-dlenv


## Model variables

In [3]:
frames = 1
channels = 1
pixels_x = 21
pixels_y = 21

## Load the model

In [4]:
model_dir = "../models/"
models_list = sorted(glob.glob(model_dir+"*.h5"))
pprint(models_list)

['../models/convlstm_1f_1c_21x_21y.h5',
 '../models/convlstm_6f_1c_21x_21y.h5',
 '../models/encoder_convlstm_1f_1c_21x_21y.h5',
 '../models/t_convlstm_1f_1c_21x_21y.h5',
 '../models/t_encoder_convlstm_1f_1c_21x_21y.h5']


In [5]:
# choose a model
file_index = 2
models_list[file_index]
model_name = 'encoder_convlstm_1f_1c_21x_21y' # todo: use regex to parse filename

In [6]:
model = tf.keras.models.load_model(models_list[file_index])
model.summary()

Model: "encodeco_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input1 (InputLayer)     [(None, 1, 1, 21, 21 0                                            
__________________________________________________________________________________________________
gaussian_noise (GaussianNoise)  (None, 1, 1, 21, 21) 0           encoder_input1[0][0]             
__________________________________________________________________________________________________
encoder1 (ConvLSTM2D)           [(None, 1, 1, 21, 21 204         gaussian_noise[0][0]             
__________________________________________________________________________________________________
decoder_input1 (InputLayer)     [(None, 1, 1, 21, 21 0                                            
_________________________________________________________________________________________

## Prepare for training

In [7]:
train_file_path = "../data/train"
valid_file_path = "../data/validate"
vars_           = ['t2m']
proc_type       = "convlstm"
# 3 years of training data = 
train_steps = 3 * 365 * 24 / frames
# 1 year of validation data = 
valid_steps = 1 * 365 * 24 / frames

In [8]:
slice_train = slice_generator(img_dir=train_file_path, slice_size=frames, vars_=vars_,
                     proc_type=proc_type, pixels_x=pixels_x, pixels_y=pixels_y, debug=False
                     )
slice_val = slice_generator(img_dir=valid_file_path, slice_size=frames, vars_=vars_,
                     proc_type=proc_type, pixels_x=pixels_x, pixels_y=pixels_y, debug=False
                     )

## Train

In [9]:
epochs = 1

In [10]:
history = model.fit(
    slice_train,
    steps_per_epoch = train_steps,
    epochs = epochs,
    verbose = 1,
    shuffle = False,
    initial_epoch = 0,
    validation_steps = valid_steps,
    validation_data = slice_val,
    )

26280/26280 [==============================] - 427s 16ms/step - loss: 80221.6016 - accuracy: 0.0353 - mean_absolute_error: 279.4272 - val_loss: 75527.1875 - val_accuracy: 0.0361 - val_mean_absolute_error: 263.7203


## Save trained model

In [11]:
tf.keras.models.save_model(
    model = model,
    filepath = '../models/t_'+model_name+'.h5',
    overwrite=True,
    include_optimizer=True,
    save_format='tf',
    signatures=None
)

## Visually Inspect Prediction

In [15]:
in_, out_ = next(slice_train)
in_

[array([[[[[284.39142, 284.29767, 284.01074, 282.7744 , 280.9188 ,
            278.5281 , 275.47937, 273.76465, 274.13354, 276.77634,
            280.4657 , 282.25842, 282.32132, 278.89136, 276.1436 ,
            273.51257, 273.192  , 272.25488, 272.0713 , 273.34637,
            274.05942],
           [284.36786, 284.32684, 284.3993 , 283.7272 , 282.53412,
            280.62177, 277.63538, 275.63937, 275.0083 , 277.31927,
            279.81888, 281.58862, 281.90527, 281.38538, 279.08673,
            275.80725, 274.8522 , 273.02017, 271.39923, 270.85013,
            273.86966],
           [284.5419 , 284.43018, 284.31113, 284.18423, 283.46164,
            281.77222, 278.81332, 276.7859 , 275.9931 , 277.37372,
            279.23105, 281.48138, 284.60254, 284.1528 , 282.1338 ,
            279.11032, 276.32492, 273.56927, 271.53174, 271.36386,
            271.46548],
           [284.58456, 284.5711 , 284.34427, 284.28586, 283.57507,
            281.8643 , 279.64932, 277.9537 , 277.39114, 2

### Test prediction

In [13]:
model.predict(in_[0], verbose=1)

1/1 [==============================] - 0s 1ms/step


array([[[[[ 0.7615942,  0.7615942,  0.7615942,  0.7615942,  0.7615942,
            0.7615942,  0.7615942,  0.7615942,  0.7615942,  0.7615942,
            0.7615942,  0.7615942,  0.7615942,  0.7615942,  0.7615942,
            0.7615942,  0.7615942,  0.7615942,  0.7615942,  0.7615942,
            0.       ],
          [ 0.7615942,  0.7615942,  0.       ,  0.       ,  0.       ,
            0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
            0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
            0.       ,  0.       ,  0.       ,  0.       ,  0.7615942,
            0.       ],
          [ 0.       ,  0.7615942,  0.7615942,  0.7615942,  0.7615942,
            0.7615942,  0.7615942,  0.7615942,  0.7615942,  0.7615942,
            0.7615942,  0.7615942,  0.7615942,  0.7615942,  0.7615942,
            0.7615942,  0.7615942,  0.7615942,  0.7615942,  0.7615942,
            0.7615942],
          [ 0.       ,  0.7615942,  0.7615942,  0.7615942,  0.7615942,
     